In [5]:
!git clone https://github.com/AceFroster/Movie-Recommendation.git


Cloning into 'Movie-Recommendation'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 1), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 1.20 MiB | 3.37 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [6]:
!ls /content/

Movie-Recommendation  sample_data


In [9]:
!ls /content/Movie-Recommendation/

Dataset  Movie.ipynb


In [10]:
!ls /content/Movie-Recommendation/Dataset/


movies.csv


In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
df = pd.read_csv('/content/Movie-Recommendation/Dataset/movies.csv')

In [32]:
df.head()

,Unnamed: 0,id,title,overview,release_date,popularity,vote_average,vote_count
0,0,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",1995-10-20,18.433,8.7,2763
1,1,724089,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,2020-07-31,8.439,8.7,1223
2,2,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,65.570,8.7,18637
3,3,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,63.277,8.7,14052
4,4,761053,Gabriel's Inferno Part III,The final part of the film adaption of the ero...,2020-11-19,26.691,8.7,773


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8551 entries, 0 to 8550
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    8551 non-null   int64  
 1   id            8551 non-null   int64  
 2   title         8551 non-null   object 
 3   overview      8531 non-null   object 
 4   release_date  8551 non-null   object 
 5   popularity    8551 non-null   float64
 6   vote_average  8551 non-null   float64
 7   vote_count    8551 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 534.6+ KB


In [34]:
df.shape

(8551, 8)

In [35]:
df.isna().sum()

,0
Unnamed: 0,0
id,0
title,0
overview,20
release_date,0
popularity,0
vote_average,0
vote_count,0


In [36]:
df2 = df.dropna()
df2.isna().sum()

,0
Unnamed: 0,0
id,0
title,0
overview,0
release_date,0
popularity,0
vote_average,0
vote_count,0


In [38]:
df2.duplicated().sum()

0

In [39]:
df2 = df2.drop(columns=['Unnamed: 0'])
df2.head()

,id,title,overview,release_date,popularity,vote_average,vote_count
0,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",1995-10-20,18.433,8.7,2763
1,724089,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,2020-07-31,8.439,8.7,1223
2,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,65.570,8.7,18637
3,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,63.277,8.7,14052
4,761053,Gabriel's Inferno Part III,The final part of the film adaption of the ero...,2020-11-19,26.691,8.7,773


In [40]:
df2.describe()

,id,popularity,vote_average,vote_count
count,8531.000000,8531.000000,8531.000000,8531.000000
mean,135447.477787,26.919045,6.580061,1499.595124
std,182242.449440,166.798457,0.824039,2466.977742
min,5.000000,0.600000,2.200000,200.000000
25%,9681.500000,10.063500,6.000000,320.000000
50%,22345.000000,13.232000,6.600000,598.000000
75%,261383.500000,20.837000,7.200000,1443.000000
max,793723.000000,11701.435000,8.700000,28740.000000


In [41]:
df2['release_date'] = pd.to_datetime(df['release_date'], errors = 'coerce')

In [42]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8531 entries, 0 to 8550
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            8531 non-null   int64         
 1   title         8531 non-null   object        
 2   overview      8531 non-null   object        
 3   release_date  8531 non-null   datetime64[ns]
 4   popularity    8531 non-null   float64       
 5   vote_average  8531 non-null   float64       
 6   vote_count    8531 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 791.2+ KB


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df2['overview'] = df2['overview'].fillna('')

tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(df2['overview'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("Cosine Similarity Matrix Shape:", cosine_sim.shape)

Cosine Similarity Matrix Shape: (8531, 8531)


In [44]:
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()

def recommend_movies(title, cosine_sim=cosine_sim, df=df2, top_n=10):
    idx = indices.get(title, None)
    if idx is None:
        return "Movie not found!"

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:top_n+1]

    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices].tolist()

print(recommend_movies("Inception"))

['New World', 'Suicide Squad: Hell to Pay', 'What Ever Happened to Baby Jane?', 'House', 'Inception: The Cobol Job', 'Cypher', 'The Wrong Missy', 'Mission: Impossible - Rogue Nation', 'Central Intelligence', 'A History of Violence']
